In [2]:
import pandas as pd
import numpy as np
import surprise
import datetime as dt
from surprise import Reader
from surprise import Dataset
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import KNNBasic
from surprise.prediction_algorithms import *

In [4]:
df =  pd.read_csv('./Data/beer_reviews.csv')

In [5]:
df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   brewery_id          1586614 non-null  int64  
 1   brewery_name        1586599 non-null  object 
 2   review_time         1586614 non-null  int64  
 3   review_overall      1586614 non-null  float64
 4   review_aroma        1586614 non-null  float64
 5   review_appearance   1586614 non-null  float64
 6   review_profilename  1586266 non-null  object 
 7   beer_style          1586614 non-null  object 
 8   review_palate       1586614 non-null  float64
 9   review_taste        1586614 non-null  float64
 10  beer_name           1586614 non-null  object 
 11  beer_abv            1518829 non-null  float64
 12  beer_beerid         1586614 non-null  int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


In [18]:
df.isna().sum()

brewery_id                0
brewery_name             15
review_time               0
review_overall            0
review_aroma              0
review_appearance         0
review_profilename      348
beer_style                0
review_palate             0
review_taste              0
beer_name                 0
beer_abv              67785
beer_beerid               0
dtype: int64

In [19]:
df['review_profilename'].value_counts()

northyorksammy    5817
BuckeyeNation     4661
mikesgroove       4617
Thorpe429         3518
womencantsail     3497
                  ... 
TedL                 1
cdfairchild31        1
JBeer18              1
1996StrokerKid       1
Ooji                 1
Name: review_profilename, Length: 33387, dtype: int64

In [20]:
df['review_profilename'].value_counts().mean()

47.511486506724175

In [21]:
df['beer_style'].value_counts()

American IPA                        117586
American Double / Imperial IPA       85977
American Pale Ale (APA)              63469
Russian Imperial Stout               54129
American Double / Imperial Stout     50705
                                     ...  
Gose                                   686
Faro                                   609
Roggenbier                             466
Kvass                                  297
Happoshu                               241
Name: beer_style, Length: 104, dtype: int64

In [22]:
df['beer_name'].value_counts()

90 Minute IPA                          3290
India Pale Ale                         3130
Old Rasputin Russian Imperial Stout    3111
Sierra Nevada Celebration Ale          3000
Two Hearted Ale                        2728
                                       ... 
Dunraven Ale                              1
Chalk Hill Gold                           1
Tripel Double                             1
Van Winkle Kristal-Weizen                 1
Schlafly Pre-Prohibition Pilsner          1
Name: beer_name, Length: 56857, dtype: int64

In [23]:
df['review_overall'].value_counts()

4.0    582764
4.5    324385
3.5    301817
3.0    165644
5.0     91320
2.5     58523
2.0     38225
1.5     12975
1.0     10954
0.0         7
Name: review_overall, dtype: int64

In [24]:
df['review_profilename'].value_counts()

northyorksammy    5817
BuckeyeNation     4661
mikesgroove       4617
Thorpe429         3518
womencantsail     3497
                  ... 
TedL                 1
cdfairchild31        1
JBeer18              1
1996StrokerKid       1
Ooji                 1
Name: review_profilename, Length: 33387, dtype: int64

In [25]:
df.describe()

,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid
count,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.518829e+06,1.586614e+06
mean,3.130099e+03,1.224089e+09,3.815581e+00,3.735636e+00,3.841642e+00,3.743701e+00,3.792860e+00,7.042387e+00,2.171279e+04
std,5.578104e+03,7.654427e+07,7.206219e-01,6.976167e-01,6.160928e-01,6.822184e-01,7.319696e-01,2.322526e+00,2.181834e+04
min,1.000000e+00,8.406720e+08,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02,3.000000e+00
25%,1.430000e+02,1.173224e+09,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,5.200000e+00,1.717000e+03
50%,4.290000e+02,1.239203e+09,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,6.500000e+00,1.390600e+04
75%,2.372000e+03,1.288568e+09,4.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,8.500000e+00,3.944100e+04
max,2.800300e+04,1.326285e+09,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.770000e+01,7.731700e+04


In [26]:
df['beer_abv'].max()

57.7

In [27]:
df[df['beer_abv']==57.7]

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
12919,6513,Schorschbräu,1316780901,4.0,4.0,4.0,kappldav123,Eisbock,4.0,3.5,Schorschbräu Schorschbock 57%,57.7,73368


In [28]:
df[df['review_profilename']== 'womencantsail']

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
216,1075,Caldera Brewing Company,1261976212,5.0,4.0,4.0,womencantsail,American Pale Ale (APA),4.0,4.5,Caldera Pale Ale,5.5,25414
489,1075,Caldera Brewing Company,1260411055,4.5,5.0,4.5,womencantsail,American IPA,3.5,4.5,Caldera IPA,6.1,10784
859,1075,Caldera Brewing Company,1269322876,3.5,3.5,4.0,womencantsail,American Amber / Red Ale,3.5,3.5,Ashland Amber,5.4,10785
922,2724,Pacific Coast Brewing Company,1293558756,3.0,3.0,3.5,womencantsail,American Strong Ale,3.5,3.0,Holiday XXII Imperial Blue,9.0,64800
927,2724,Pacific Coast Brewing Company,1293559076,1.0,1.5,3.0,womencantsail,American Strong Ale,2.5,1.5,Megalodon,NaN,64803
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1585933,3835,Drake's Brewing Co.,1304806741,2.0,2.5,4.0,womencantsail,American Wild Ale,3.5,2.5,Drake's Simian Sour,9.0,68807
1585957,3835,Drake's Brewing Co.,1293742192,5.0,4.5,4.0,womencantsail,American Pale Ale (APA),4.0,4.5,Alpha Session,3.8,63994
1586005,3835,Drake's Brewing Co.,1258773000,3.5,4.0,4.0,womencantsail,American Pale Ale (APA),3.5,3.5,Drake's 1500 Pale Ale,5.2,25095
1586042,3835,Drake's Brewing Co.,1322844943,3.5,3.5,4.0,womencantsail,Saison / Farmhouse Ale,4.0,3.5,Pink Boots Saison,6.0,75910


In [29]:
df2 = df.groupby('review_profilename').filter(lambda x : len(x)>10)
df2 = df2.groupby('review_profilename').filter(lambda x : len(x)<1000)

In [30]:
df2['review_profilename'].value_counts()

david18           996
junjun            994
BDTyre            989
DaPeculierDane    987
donkeyrunner      985
                 ... 
toastdog           11
tanstaafl28        11
tfuchs             11
erok               11
edgeofcenter       11
Name: review_profilename, Length: 9936, dtype: int64

In [31]:
df2.isna().sum()

brewery_id                0
brewery_name             11
review_time               0
review_overall            0
review_aroma              0
review_appearance         0
review_profilename        0
beer_style                0
review_palate             0
review_taste              0
beer_name                 0
beer_abv              37790
beer_beerid               0
dtype: int64

In [32]:
df2['beer_name'].value_counts()

90 Minute IPA                          2668
Old Rasputin Russian Imperial Stout    2580
Sierra Nevada Celebration Ale          2536
Two Hearted Ale                        2269
Stone Ruination IPA                    2256
                                       ... 
Afterglow IPA                             1
Hix Brown Ale                             1
Chocolate Covered Cherry Stout            1
Eis Instigator                            1
Schlafly Pre-Prohibition Pilsner          1
Name: beer_name, Length: 44035, dtype: int64

In [33]:
df2 = df2.groupby('beer_name').filter(lambda x : len(x)>200)

In [34]:
df2['beer_style'].value_counts()

American IPA                        58494
American Double / Imperial IPA      46308
Russian Imperial Stout              31026
American Double / Imperial Stout    26610
American Pale Ale (APA)             20967
                                    ...  
Euro Strong Lager                     237
Faro                                  234
English Stout                         222
Smoked Beer                           219
American Dark Wheat Ale                 8
Name: beer_style, Length: 96, dtype: int64

In [35]:
df2['beer_name'].value_counts()

90 Minute IPA                             2668
Old Rasputin Russian Imperial Stout       2580
Sierra Nevada Celebration Ale             2536
Two Hearted Ale                           2269
Stone Ruination IPA                       2256
                                          ... 
Bell's Lager Of The Lakes                  201
Andygator                                  201
Samuel Adams Black & Brew Coffee Stout     201
Blue Star Great American Wheat Beer        201
Strawberry Harvest Lager                   201
Name: beer_name, Length: 1236, dtype: int64

In [36]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 634964 entries, 10 to 1586587
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   brewery_id          634964 non-null  int64  
 1   brewery_name        634964 non-null  object 
 2   review_time         634964 non-null  int64  
 3   review_overall      634964 non-null  float64
 4   review_aroma        634964 non-null  float64
 5   review_appearance   634964 non-null  float64
 6   review_profilename  634964 non-null  object 
 7   beer_style          634964 non-null  object 
 8   review_palate       634964 non-null  float64
 9   review_taste        634964 non-null  float64
 10  beer_name           634964 non-null  object 
 11  beer_abv            633079 non-null  float64
 12  beer_beerid         634964 non-null  int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 67.8+ MB


In [37]:
df2.columns

Index(['brewery_id', 'brewery_name', 'review_time', 'review_overall',
       'review_aroma', 'review_appearance', 'review_profilename', 'beer_style',
       'review_palate', 'review_taste', 'beer_name', 'beer_abv',
       'beer_beerid'],
      dtype='object')

In [38]:
#df2.to_csv('./Data/data_viz.csv')

In [39]:
df2.drop(columns = ['brewery_id', 'brewery_name', 'review_aroma', 'review_appearance', 'beer_style','review_palate','review_taste','beer_abv','beer_beerid','review_time'], inplace = True)

In [40]:
df2.columns

Index(['review_overall', 'review_profilename', 'beer_name'], dtype='object')

In [41]:
df2.head()

,review_overall,review_profilename,beer_name
10,3.0,fodeeoz,Amstel Light
30,3.0,xXTequila,Amstel Light
265,4.0,coldriver,Caldera IPA
337,4.0,Rutager,Caldera IPA
338,4.5,CHILLINDYLAN,Caldera IPA


In [42]:
df2 = df2.rename(columns={'review_profilename': 'user', 'beer_name': 'item', 'review_overall': 'rating'})

In [43]:
df2.head()

,rating,user,item
10,3.0,fodeeoz,Amstel Light
30,3.0,xXTequila,Amstel Light
265,4.0,coldriver,Caldera IPA
337,4.0,Rutager,Caldera IPA
338,4.5,CHILLINDYLAN,Caldera IPA


In [44]:
df2 = df2[[ 'user', 'item', 'rating']]

In [45]:
df2.head()

,user,item,rating
10,fodeeoz,Amstel Light,3.0
30,xXTequila,Amstel Light,3.0
265,coldriver,Caldera IPA,4.0
337,Rutager,Caldera IPA,4.0
338,CHILLINDYLAN,Caldera IPA,4.5


In [47]:
#df2.to_csv('./Data/clean_data2.csv')

In [ ]:
reader = Reader(line_format=u'rating user item', sep=',', rating_scale=(1, 5), skip_lines=1)


In [ ]:
data = Dataset.load_from_df(df2[["user", "item", "rating"]], reader)

In [ ]:
train, test = surprise.model_selection.train_test_split(data, random_state=42)

In [ ]:
model = KNNBasic().fit(train)

In [ ]:
model2 = SVD().fit(train)

In [ ]:
model3 = NMF().fit(train)

In [ ]:
model.test(test)

In [ ]:
surprise.accuracy.mae(model.test(test))

In [ ]:
surprise.accuracy.mae(model2.test(test))

In [ ]:
surprise.accuracy.mae(model3.test(test))

In [ ]:
surprise.accuracy.rmse(model.test(test))

In [ ]:
surprise.accuracy.rmse(model2.test(test))

In [ ]:
surprise.accuracy.rmse(model3.test(test))

In [ ]:
predictions=model2.test(test)

In [ ]:
predictions[0]

In [ ]:
predictions[1]